In [0]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
# layers
from tensorflow.keras.layers import Input, Dense, Flatten, SimpleRNN, GRU, LSTM, GlobalMaxPooling1D
# model
from tensorflow.keras.models import Sequential
# optimizers
from tensorflow.keras.optimizers import Adam, SGD

# additional imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# Function to generate labels 
def get_label(x, i1, i2, i3):
  # x = sequence
  if x[i1] < 0 and x[i2] < 0 and x[i3] < 0:
    return 1
  if x[i1] < 0 and x[i2] > 0 and x[i3] > 0:
    return 1
  if x[i1] > 0 and x[i2] < 0 and x[i3] > 0:
    return 1
  if x[i1] > 0 and x[i2] > 0 and x[i3] < 0:
    return 1
  return 0

In [0]:
# Function to generate data
def get_data(T, D, L = 5000, data_type = 'long'):
  X = []
  Y = []
  for t in range(L):
    x = np.random.randn(T)
    X.append(x)
    if data_type == 'long':
      y = get_label(x, 1, 2, 3)
    elif data_type == 'short':
      y = get_label(x, -1, -2, -3)
    Y.append(y)
  X = np.array(X)
  Y = np.array(Y)
  N = len(X)
  return X,Y,N

In [0]:
# Function to visualise results
def visualise(r, title = 'Graph'):
  plt.subplot(2,1)
  plt.plot(r.history['loss'], label = title + ' loss')
  plt.plot(r.history['val_loss'], label = title + ' val_loss')
  plt.legend()
  plt.subplot(2,2)
  plt.plot(r.history['accuracy'], label = title + ' accuracy')
  plt.plot(r.history['val_accuracy'], label = title + ' val_accuracy')
  plt.legend()
  plt.show()

In [0]:
# Lets start with T=20
T = 20
D = 1
X, Y, N = get_data(T, D, L = 5000, data_type = 'long')
# expand the dimensions of X to pass it to RNN
X = np.expand_dims(X, -1)
print(X.shape, Y.shape)

(5000, 20, 1) (5000,)


In [0]:
# function to do experimentation models automatically
def test(layer_sets, optimizer, optimizer_name, loss, metrics, X,Y, epochs = 200, validation_split = .5):
  result = pd.DataFrame()
  histories = {}
  for layer_set in layer_sets:
    m = Sequential(layer_set['layers'])
    m.compile(loss = loss, optimizer = optimizer, metrics = metrics)
    r = m.fit(X,Y, validation_split = validation_split, epochs = epochs, verbose = 1)
    histories[layer_set['name']] = r
    results = [layer_set['name'],
               r.history['loss'][-1],
               r.history['val_loss'][-1],
               r.history['accuracy'][-1],
               r.history['val_accuracy'][-1],
               optimizer_name,
               loss,
               epochs,
               validation_split]
    result = pd.concat((result, pd.DataFrame([pd.Series(results)])))
  cols = ['Name', 'Loss', 'Validation Loss', 'Accuracy', 'Validation Accuracy', 'Optimizer', 'Loss Function', '# of Epochs', 'Data Split Rate For Validation']
  result.columns = cols
  return result, histories

In [11]:
# try to do simple experimentation with LSTM
lstm = {'name': 'LSTM',
        'layers': [
                   Input(shape = (T,D)),
                   LSTM(10),
                   Dense(1, activation = 'sigmoid')
        ]}
stacked_lstm = {'name': '3 Stacked LSTM',
        'layers': [
                   Input(shape = (T,D)),
                   LSTM(10, return_sequences = True),
                   LSTM(10, return_sequences = True),
                   LSTM(10),
                   Dense(1, activation = 'sigmoid')
        ]}
max_pooling_lstm = {'name': 'Max Pooling LSTM',
        'layers': [
                   Input(shape = (T,D)),
                   LSTM(10, return_sequences = True),
                   GlobalMaxPooling1D(),
                   Dense(1, activation = 'sigmoid')
        ]}
gru = {'name': 'GRU',
        'layers': [
                   Input(shape = (T,D)),
                   GRU(10),
                   Dense(1, activation = 'sigmoid')
        ]}
stacked_gru = {'name': '3 Stacked GRU',
        'layers': [
                   Input(shape = (T,D)),
                   GRU(10, return_sequences=True),
                   GRU(10, return_sequences=True),
                   GRU(10),
                   Dense(1, activation = 'sigmoid')
        ]}
max_pooling_gru = {'name': 'Max Pooling GRU',
        'layers': [
                   Input(shape = (T,D)),
                   GRU(10, return_sequences = True),
                   GlobalMaxPooling1D(),
                   Dense(1, activation = 'sigmoid')
        ]}

subjects = [lstm, stacked_lstm, max_pooling_lstm, gru, stacked_gru, max_pooling_gru]

test_results, histories = test(layer_sets = subjects, 
                   optimizer_name =  'Adam',
                   optimizer = Adam(lr=0.05),
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'],
                   X = X,
                   Y = Y,
                   epochs = 100)

Train on 2500 samples, validate on 2500 samples
Epoch 1/100
2500/2500 [==============================] - 3s 1ms/sample - loss: 0.6953 - accuracy: 0.5028 - val_loss: 0.6936 - val_accuracy: 0.5052
Epoch 2/100
2500/2500 [==============================] - 1s 328us/sample - loss: 0.6950 - accuracy: 0.5044 - val_loss: 0.6975 - val_accuracy: 0.4900
Epoch 3/100
2500/2500 [==============================] - 1s 319us/sample - loss: 0.6935 - accuracy: 0.5076 - val_loss: 0.6943 - val_accuracy: 0.4900
Epoch 4/100
2500/2500 [==============================] - 1s 324us/sample - loss: 0.6952 - accuracy: 0.4980 - val_loss: 0.6933 - val_accuracy: 0.5100
Epoch 5/100
2500/2500 [==============================] - 1s 318us/sample - loss: 0.6941 - accuracy: 0.4912 - val_loss: 0.6930 - val_accuracy: 0.5100
Epoch 6/100
2500/2500 [==============================] - 1s 336us/sample - loss: 0.6948 - accuracy: 0.5060 - val_loss: 0.6945 - val_accuracy: 0.4892
Epoch 7/100
2500/2500 [==============================] - 1s 

In [12]:
test_results

,Name,Loss,Validation Loss,Accuracy,Validation Accuracy,Optimizer,Loss Function,# of Epochs,Data Split Rate For Validation
0,LSTM,0.689043,0.707579,0.5324,0.4868,Adam,binary_crossentropy,100,0.5
0,3 Stacked LSTM,0.695979,0.692971,0.5012,0.5100,Adam,binary_crossentropy,100,0.5
0,Max Pooling LSTM,0.018176,0.019642,0.9968,0.9940,Adam,binary_crossentropy,100,0.5
0,GRU,0.693685,0.693233,0.4904,0.5076,Adam,binary_crossentropy,100,0.5
0,3 Stacked GRU,0.698541,0.693407,0.4812,0.5100,Adam,binary_crossentropy,100,0.5
0,Max Pooling GRU,0.695728,0.693493,0.5056,0.5052,Adam,binary_crossentropy,100,0.5
